Connect Colab notebook to Google Drive and git clone the forambulator project

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')
home = '/content/drive/My Drive/Forams'
os.chdir(home)
!git clone https://github.com/metazool/forambulator

Optionally pull any updates from git if the drive is already mounted

In [2]:
import os, sys
home = '/content/drive/My Drive/Forams'
os.chdir(home)
os.chdir('forambulator')
!git pull
!git checkout stylegan2
os.chdir(home)
import sys
sys.path.append('./forambulator')

Already up to date.
Already on 'stylegan2'
Your branch is up to date with 'origin/stylegan2'.


**Run only once:** collect images from Endless Forams. They'll end up in subdirectories of `./data` matching the taxonomic label for each set. We keep the metadata around for good measure

In [0]:
import sys
sys.path.append('forambulator')
from forams.download import download_capsules
download_capsules()

**Run only once:** Given a collection of Endless Forams images in `./data`, make a best-faith attempt to extract only the regions containing the foram, leaving a set of 256*256 images in `DIRECTORY`

In [0]:
from forams.images import list_image_filenames, crop_foram, NoForamFound
images = list_image_filenames('./data')
DIRECTORY = 'cropped_256'
for image in images:
    try:
        crop_foram(image, directory=DIRECTORY)
    except NoForamFound:
        pass

**Run only once**: convert the image dataset into `tfrecords` form for use by the network


In [0]:
from forams.images import tfrecords_from_images
os.mkdir('tfrecords')
tfrecords_from_images('./tfrecords','./cropped_256', True)

Check out the stylegan2 repository and collect any updates


In [4]:
import os, sys
!git clone https://github.com/NVLabs/stylegan2
os.chdir('stylegan2')
!git pull
os.chdir(home)
sys.path.append('stylegan2')

fatal: destination path 'stylegan2' already exists and is not an empty directory.
Already up to date.


Start training the model. If we have a previous colab run to restart from, provide its filename.

In [0]:
import sys
sys.path.append('stylegan2')

# resume_from_filename = os.path.join(home, 'results/00008-sgan-custom-1gpu/network-snapshot-003045.pkl')
resume_from_filename = None

import forams.train
forams.train.train(data_dir=home,
                   dataset='tfrecords', 
                   metrics=[],
                   resume_from=resume_from_filename,
                   save_ticks=2)


Local submit - run_dir: results/00014-stylegan2-tfrecords-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...


Streaming data using training.dataset.TFRecordDataset...


INFO:numexpr.utils:NumExpr defaulting to 4 threads.


Dataset shape = [1, 256, 256]
Dynamic range = [0, 255]
Label size    = 0
Constructing networks...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.

G                             Params    OutputShape         WeightShape     
---                           ---       ---                 ---             
latents_in                    -         (?, 512)            -               
labels_in                     -         (?, 0)              -               
lod                           -         ()                  -               
dlatent_avg                   -         (512,)              -               
G_mapping/latents_in          -         (?, 512)            -               
G_mapping/labels_in           -         (?, 0)              -               
G_mapping/Normalize           -         (?, 512)            -               
G_mapping/Dense0              262656    (?, 